In [1]:
import os
import sys

sys.path.append("../../common/src")
sys.path.append("../src")
os.chdir("../src")

In [2]:
from langchain.agents import Tool, AgentExecutor, BaseMultiActionAgent, AgentOutputParser
from langchain.agents.chat.base import ChatAgent
from langchain.prompts import StringPromptTemplate
from langchain.llms import OpenAI, VertexAI
from langchain.chat_models import ChatVertexAI, ChatOpenAI
from langchain.utilities import SerpAPIWrapper
from langchain.chains import LLMChain
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish, OutputParserException
import re

In [3]:
!export PROJECT_ID="gcp-mira-demo"
project = "gcp-mira-demo"
os.environ["PROJECT_ID"] = project
PROJECT_ID = project

In [4]:
import config
from config import (VERTEX_LLM_TYPE_BISON_CHAT,
                    VERTEX_LLM_TYPE_BISON_TEXT,
                    OPENAI_LLM_TYPE_GPT3_5,
                    OPENAI_LLM_TYPE_GPT4,
                    COHERE_LLM_TYPE,
                    LLM_BACKEND_ROBOT_USERNAME,
                    LLM_BACKEND_ROBOT_PASSWORD)
from common.utils.token_handler import UserCredentials
config.TOOLS_SERVICE_BASE_URL = "https://gcp-mira-demo.cloudpssolutions.com/tools-service/api/v1"
config.auth_client = UserCredentials(LLM_BACKEND_ROBOT_USERNAME,
                              LLM_BACKEND_ROBOT_PASSWORD,
                              "https://gcp-mira-demo.cloudpssolutions.com")

INFO: [config/config.py:54 - <module>()] Namespace File not found, setting job namespace as default
INFO: [config/config.py:87 - get_environ_flag()] ENABLE_GOOGLE_LLM = True
INFO: [config/config.py:87 - get_environ_flag()] ENABLE_GOOGLE_MODEL_GARDEN = True
INFO: [config/config.py:87 - get_environ_flag()] ENABLE_LLAMA2CPP_LLM = False
INFO: [config/config.py:87 - get_environ_flag()] ENABLE_OPENAI_LLM = True
INFO: [config/config.py:87 - get_environ_flag()] ENABLE_COHERE_LLM = True
INFO: [config/config.py:87 - get_environ_flag()] ENABLE_TRUSS_LLAMA2 = True
INFO: [config/config.py:246 - <module>()] Can't load llm_service_models.json: 404 Secret [projects/63101149566/secrets/llm_service_password_Llama2cpp] not found or has no versions. [detail: "[ORIGINAL ERROR] generic::not_found: Secret [projects/63101149566/secrets/llm_service_password_Llama2cpp] not found or has no versions. [google.rpc.error_details_ext] { message: \"Secret [projects/63101149566/secrets/llm_service_password_Llama2cpp] n

In [5]:
config.SERVICES['tools-service']['host']='https://gcp-mira-demo.cloudpssolutions.com'

In [6]:
config.SERVICES['tools-service']['api_url_prefix']='https://gcp-mira-demo.cloudpssolutions.com/tools-service/api/v1'

In [7]:
config.SERVICES['tools-service']

{'host': 'https://gcp-mira-demo.cloudpssolutions.com',
 'port': 80,
 'api_path': '/tools-service/api/v1',
 'api_url_prefix': 'https://gcp-mira-demo.cloudpssolutions.com/tools-service/api/v1'}

In [8]:
from common.models import Agent
from common.models.agent import AgentType, UserPlan, PlanStep
from services.agents.agent_service import get_agent_config, get_plan_agent_config,agent_execute_plan, run_agent

In [9]:
from services.agents.agents import TaskAgent, PlanAgent
#casey_plan_agent = TaskAgent(VERTEX_LLM_TYPE_BISON_TEXT)
#casey_plan_agent = TaskAgent(VERTEX_LLM_TYPE_BISON_CHAT)
#casey_plan_agent = TaskPlanAgent(OPENAI_LLM_TYPE_GPT3_5)
#casey_plan_agent = TaskAgent(OPENAI_LLM_TYPE_GPT4)
casey_plan_agent = TaskAgent(OPENAI_LLM_TYPE_GPT4)
#casey_plan_agent = TaskPlanAgent(COHERE_LLM_TYPE)

In [10]:
prompt = """Compose and send an email to all the medicaid applicants that are missing income verification asking them to provide a pay stub from their employers"""

In [11]:
user_id = "5nJrkPWa3D0yCKA853mD"

In [12]:
#agent_executor.run(agent_inputs)

In [13]:
from services.agents.agent_service import agent_plan, parse_plan

In [14]:
plan = agent_plan("Plan", prompt, user_id)

user_plan = plan[1]
plan_steps=user_plan.plan_steps
chat_history = []

for step in plan_steps:
    description = PlanStep.find_by_id(step).description
    chat_history.append(description)

INFO: [agents/agent_service.py:178 - agent_plan()] Starting with plan for agent_name=[Plan], prompt=[Compose and send an email to all the medicaid applicants that are missing income verification asking them to provide a pay stub from their employers], user_id=[5nJrkPWa3D0yCKA853mD], chat_history=[None]
INFO: [agents/agent_service.py:136 - run_agent()] Running Plan agent with prompt=[Compose and send an email to all the medicaid applicants that are missing income verification asking them to provide a pay stub from their employers] and chat_history=[None]
INFO: [agents/agent_service.py:145 - run_agent()] Available tools=[gmail_tool, docs_tool, calendar_tool, search_tool, query_tool]
INFO: [agents/agents.py:104 - load_agent()] Successfully loaded PlanAgent agent.
INFO: [agents/agent_service.py:157 - run_agent()] Running agent executor.... 
INFO: [agents/agent_service.py:159 - run_agent()] Agent Plan generated output=[The task involves identifying Medicaid applicants who are missing income

In [15]:
tools = casey_plan_agent.get_tools()
print(tools)

agent = casey_plan_agent.load_agent()

#prompt = """You are an AI assistant that can execute steps by calling upon the right tools. You have access to all the information required to execute the plan by from the values returned by the tools . Execute the steps as outlined in the following numbered list """
prompt = "Execute the steps provided"
agent_executor = AgentExecutor.from_agent_and_tools(
  agent=agent, 
    tools=tools,
    verbose=True)




[StructuredTool(name='gmail_tool', description='gmail_tool(recipients: List, subject: str, message: str) -> str - Send an email to a list of recipients', args_schema=<class 'pydantic.main.gmail_toolSchemaSchema'>, func=<function gmail_tool at 0x15c5a59d0>), StructuredTool(name='docs_tool', description='docs_tool(recipients: List, content: str) -> Dict - Compose or create a document using Google Docs', args_schema=<class 'pydantic.main.docs_toolSchemaSchema'>, func=<function docs_tool at 0x15c5a5a60>), StructuredTool(name='calendar_tool', description='calendar_tool(date: str) -> str - Create and update meetings using Google Calendar', args_schema=<class 'pydantic.main.calendar_toolSchemaSchema'>, func=<function calendar_tool at 0x15c5a5af0>), StructuredTool(name='search_tool', description='search_tool(query: str) -> str - Perform an internet search.', args_schema=<class 'pydantic.main.search_toolSchemaSchema'>, func=<function search_tool at 0x15c5a5b80>), StructuredTool(name='query_tool

In [16]:
#chat_history[0]= '1. Use [query_tool] to [ find Medicaid applicants missing a pay stub ]'
#chat_history[2] = '3. Use [gmail_tool] to [send the composed email to the identified applicants]' 
#chat_history.insert(1,'Use [docs_tool] to [compose an email asking for a pay stub from their employers]')

In [17]:
chat_history_string = ''.join(chat_history)
agent_inputs = {
    "input": prompt + chat_history_string
}
#agent_inputs = {
#    "input": prompt, 
#    'chat_history':chat_history
#}

In [18]:
agent_inputs['input']

'Execute the steps provided1. Use [query_tool] to [identify Medicaid applicants who are missing income verification]2. Use [docs_tool] to [compose an email asking the applicants to provide a pay stub from their employers]3. Use [gmail_tool] to [send the composed email to the identified applicants]'

In [19]:
result = agent_executor.run(agent_inputs)



> Entering new AgentExecutor chain...
Thought: The first step is to identify Medicaid applicants who are missing income verification. I will use the query_tool to perform this action.

Action:
```
{
  "action": "query_tool",
  "action_input": "identify Medicaid applicants who are missing income verification"
}
```
Observation: {'recipients': ['sumeetvij@google.com']}
Thought:The query_tool has returned a list of recipients who are missing income verification for Medicaid. The next step is to compose an email asking these applicants to provide a pay stub from their employers. I will use the docs_tool to compose this email.

Action:
```
{
  "action": "docs_tool",
  "action_input": {
    "recipients": ["sumeetvij@google.com"],
    "content": "Dear Applicant, \n\nWe have noticed that your Medicaid application is missing income verification. Please provide a pay stub from your employer at your earliest convenience. \n\nThank you, \nMedicaid Services"
  }
}
```[docs_tool]: ['sumeetvij@goog

In [21]:
#call agent_service that looks up the plan steps for a given User Plan
agent_execute_plan(agent_name='Task', prompt=prompt,user_plan=user_plan)

INFO: [agents/agent_service.py:243 - agent_execute_plan()] Running Task agent with prompt=[Execute the steps provided] and user_plan=[<common.models.agent.UserPlan object at 0x13030e1f0>]
INFO: [agents/agents.py:104 - load_agent()] Successfully loaded TaskAgent agent.
INFO: [agents/agent_service.py:253 - agent_execute_plan()] Available tools=[gmail_tool, docs_tool, calendar_tool, search_tool, query_tool]
INFO: [agents/agent_service.py:269 - agent_execute_plan()] Running agent executor.... input:Execute the steps provided1. Use [query_tool] to [identify Medicaid applicants who are missing income verification]2. Use [docs_tool] to [compose an email asking the applicants to provide a pay stub from their employers]3. Use [gmail_tool] to [send the composed email to the identified applicants] 
INFO: [agents/agent_tools.py:111 - docs_tool()] [docs_tool] Composed an email with subject:  Subject: Income Verification Request for Application Review
INFO: [agents/agent_tools.py:73 - gmail_tool()] 

('The steps have been successfully executed. The Medicaid applicants who were missing income verification have been identified and an email has been sent to them asking for a pay stub from their employers.',
 '\n\n\x1b> Entering new AgentExecutor chain...\x1b\n\x1b\x1bThought: The first step is to identify Medicaid applicants who are missing income verification. I will use the query_tool to perform this action.\n\nAction:\n```\n{\n  "action": "query_tool",\n  "action_input": "identify Medicaid applicants who are missing income verification"\n}\n```\x1b\nObservation: \x1b\x1b{\'recipients\': [\'sumeetvij@google.com\']}\x1b\nThought:\x1b\x1bThe query_tool has returned a list of recipients who are missing income verification for Medicaid. The next step is to compose an email asking these applicants to provide a pay stub from their employers. I will use the docs_tool to compose this email.\n\nAction:\n```\n{\n  "action": "docs_tool",\n  "action_input": {\n    "recipients": ["sumeetvij@goog

In [23]:

run_agent(agent_name='Task', prompt=agent_inputs['input'])

INFO: [agents/agent_service.py:136 - run_agent()] Running Task agent with prompt=[Execute the steps provided1. Use [query_tool] to [identify Medicaid applicants who are missing income verification]2. Use [docs_tool] to [compose an email asking the applicants to provide a pay stub from their employers]3. Use [gmail_tool] to [send the composed email to the identified applicants]] and chat_history=[None]
INFO: [agents/agent_service.py:145 - run_agent()] Available tools=[gmail_tool, docs_tool, calendar_tool, search_tool, query_tool]
INFO: [agents/agents.py:104 - load_agent()] Successfully loaded TaskAgent agent.
INFO: [agents/agent_service.py:157 - run_agent()] Running agent executor.... 
[docs_tool]: ['sumeetvij@google.com']
INFO: [agents/agent_tools.py:111 - docs_tool()] [docs_tool] Composed an email with subject:  Subject: Income Verification Request for Application Review
INFO: [agents/agent_tools.py:73 - gmail_tool()] [gmail_tool] Sending email to sumeetvij@google.com. Result: Message

'The email requesting income verification has been successfully sent to the identified Medicaid applicants.'